# Toy problem for debugging

$$
\begin{align*}
\max_{x,y} \quad & f'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y \leq d_k, \\
& y\in \{0,1\}.
\end{align*}
$$

### Example from
http://www.optirisk-systems.com/manuals/FortspManual.pdf

In [1]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
import numpy as np

$$
\begin{align*}
\max_{x,y} \quad & \sum_k P_k \left( \sum_c p^s_c x^s_{c,k} - p^b_c x^b_{c,k}  - \sum_c c_c a_c \right)\\
\mathrm{s.t.} \quad & \sum_c a_c \leq A \\
& Y_{c,k}  a_c - x^s_{c,k} + x^b_{c,k} \geq R_c \\
& x^s_{\mathrm{beet},k} \leq Q(\mathrm{beet}) \\
& x^s_{\mathrm{beet},c} + s^x_k \leq Y_{\mathrm{beet},k} a_\mathrm{beet}
\end{align*}
$$

In [2]:
from dynamicme.decomposition import BendersMaster, BendersSubmodel

In [3]:
#------------------------------------------------------------
# Data
crops = ['wheat','corn','beet']
total_area = 500.  # Total acres
prob_dict = {'below':1./3, 'average':1./3, 'above':1./3}
# Yields (tons/acre) over scenarios
yield_dict = {}
yield_dict['below'] = {'wheat':2., 'corn':2.4, 'beet':16.}
yield_dict['average'] = {'wheat':2.5, 'corn':3., 'beet':20.}
yield_dict['above'] = {'wheat':3., 'corn':3.6, 'beet':24.}
# Planting costs
plantcost_dict={'wheat':150, 'corn':230, 'beet':260}   # $/ton
sellprice_dict = {'wheat':170, 'corn':150, 'beet':36}  # $/ton
excess_sell_price = 10    # beet sell price ($/ton) when quota exceeded
beet_quota = 6000   # tons
buyprice_dict = {'wheat':238, 'corn':210, 'beet':100}  # $/ton
req_dict = {'wheat':200., 'corn':240., 'beet':0.} # tons required to feed cattle
#------------------------------------------------------------
# Variables
# y: area is complicating (first-stage) variable
area_dict0 = {c:Variable('area_%s'%c, lower_bound=0., upper_bound=10000,
                       objective_coefficient=plantcost_dict[c]) for c in crops}
for y in area_dict0.values():
    y.variable_kind = 'integer'

In [4]:
area_dict0

{'beet': <Variable area_beet at 0x7f8c3198a4d0>,
 'corn': <Variable area_corn at 0x7f8c3198a490>,
 'wheat': <Variable area_wheat at 0x7f8c3198a450>}

In [5]:
# Create subproblems
UB = 1e4
sub_dict = {}
for scen,yields in iteritems(yield_dict):
    # Need to copy master vars
    area_dict = {k:v.copy() for k,v in iteritems(area_dict0)}
    mdl = Model('sub')
    mdl.add_reactions(area_dict.values())
    # Global constraint: sum_a <= Area
    cons_area = Constraint('total_area')
    cons_area._bound = total_area
    cons_area._constraint_sense = 'L'
    mdl.add_metabolites(cons_area)
    for y in area_dict.values():
        y.add_metabolites({cons_area:1.}, combine=False)

    x_excess = Variable('sell_excess_beet', lower_bound=0., upper_bound=UB)
    x_excess.objective_coefficient = -prob_dict[scen]*excess_sell_price    
    mdl.add_reaction(x_excess)
    
    for c in crops:
        ### Scenario variables x: tons sold, bought, excess sold per scenario
        x_sell = Variable('sell_%s'%(c), lower_bound=0., upper_bound=UB)
        x_buy  =  Variable('buy_%s'%(c), lower_bound=0., upper_bound=UB)
        mdl.add_reactions([x_sell,x_buy])        
        x_sell.objective_coefficient = -prob_dict[scen]*sellprice_dict[c]
        x_buy.objective_coefficient = prob_dict[scen]*buyprice_dict[c]
        
        ### Scenario constraint: Y_ck*ac - xs_ck + xb_ck >= Req_c        
        cons = Constraint('required_%s'%c)
        cons._bound = req_dict[c]
        cons._constraint_sense = 'G'        
        mdl.add_metabolites(cons)
        area = area_dict[c]
        area.add_metabolites({cons:yield_dict[scen][c]})
        x_sell.add_metabolites({cons:-1.})
        x_buy.add_metabolites({cons:1.})
        
        ### Beet quota
        if c=='beet':
            x_sell.upper_bound = beet_quota
            # xs_beet,k + xexcess_k <= Yield_beet,k * area_beet
            cons_beet = Constraint('beet_balance')
            cons_beet._bound = 0.
            cons_beet._constraint_sense = 'L'
            x_sell.add_metabolites({cons_beet:1.})
            x_excess.add_metabolites({cons_beet:1.})
            area_dict[c].add_metabolites({cons_beet:-yield_dict[scen][c]})
    
    sub = BendersSubmodel(mdl, scen)
    sub_dict[scen] = sub

Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


In [6]:
master = BendersMaster(mdl)
master._INF = 1e6
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


In [7]:
for v in master.model.getVars():
    print(v.VarName, v.LB, v.UB)

('z', -1000000.0, 1000000.0)
('area_wheat', -0.0, 10000.0)
('area_corn', -0.0, 10000.0)
('area_beet', -0.0, 10000.0)
('tk_below', -1000000.0, 1000000.0)
('tk_average', -1000000.0, 1000000.0)
('tk_above', -1000000.0, 1000000.0)


In [8]:
cc = master.model.getConstrs()[0]
print(master.model.getRow(cc), cc.Sense, cc.RHS)

(<gurobi.LinExpr: area_wheat + area_corn + area_beet>, '<', 500.0)


In [9]:
master.model.getObjective()

<gurobi.LinExpr: z>

In [10]:
cz = master.model.getConstrByName('z_cut')
print(master.model.getRow(cz), cz.Sense, cz.RHS)

(<gurobi.LinExpr: z + -150.0 area_wheat + -230.0 area_corn + -260.0 area_beet + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '=', 0.0)


In [11]:
from dynamicme.callback_gurobi import cb_benders_multi

# Solve using callbacks

In [12]:
# master.model.optimize(cb_benders_multi)

# Solve without callbacks

In [13]:
def opt_nondom(sub, y0):
    # Add or update the constraint for same obj
    cons = sub.model.model.getConstrByName('fixobjval')
    if cons is None:
        expr = sub.model.model.getObjective() == sub.model.model.ObjVal
        cons = sub.model.model.addConstr(expr, name='fixobjval')
    else:
        cons.RHS = sub.model.model.ObjVal
        obj = sub.model.model.getObjective()
        for j in range(obj.size()):
            v = obj.getVar(j)
            sub.model.model.chgCoeff(cons, v, obj.getCoeff(j))
    # Change the objective function
    sub.update_obj(y0)
    sub.model.optimize()
    # Relax the constraint for next iter
    cons.Sense = GRB.LESS_EQUAL
    cons.RHS   = GRB.INFINITY

In [14]:
LP_RELAXATION = False
NONDOM = False
MULTICUT = True

y0 = np.ones(len(area_dict))

from gurobipy import GRB, LinExpr, quicksum
import gurobipy as grb

from __future__ import division
import time
master.precision_sub = 'gurobi'
master.verbosity = 1
master.print_iter = 1

master.model.Params.OutputFlag = 0
master.model.Params.TimeLimit = 3600*2
master.model.Params.Presolve = 0
### LP Relaxation
if LP_RELAXATION:
    print('Performing LP relaxation of Master')
    for v in master.model.getVars():
        v.VType = GRB.CONTINUOUS
###--------------
maxiter = 2000
UB = 1e15
LB =-1e15
gap= UB-LB
relgap = gap / (1e-10+abs(UB))  

bestUB = UB
bestLB = LB
yopt   = y0
tic    = time.time()

print("%-12.10s%-12.8s%-12.8s%-12.8s%-12.8s%-12.8s%-12.10s%-12.8s" % (
    'Iter','UB','LB','Best UB','Best LB','gap','relgap(%)','time(s)'))
    #_iter,UB,LB,bestUB,bestLB,gap,relgap*100,toc))

for _iter in range(maxiter):
    if np.mod(_iter,master.print_iter)==0:
        toc = time.time()-tic
        #print("iter=%d\tUB=%g\tLB=%g\tBest UB=%g\tBest LB=%g\tgap=%g\trelgap=%.4g%%\ttime=%g (s)" % (
        print("%-12.10s%-12.4g%-12.4g%-12.4g%-12.4g%-12.4g%-12.4g%-12.10g" % (
            _iter,UB,LB,bestUB,bestLB,gap,relgap*100,toc))
        
    master.model.optimize()    
    fy = master._fy
    ys = master._ys
    z  = master._z
    yprev = np.array(yopt)
    yopt = [y.X for y in ys]
    sub_dict = master.sub_dict
    sub_objs = []
    opt_sub_inds = []
    
    ### Update core point
    y0 = 0.5*(y0 + yopt)
    y0[y0<master.model.Params.IntFeasTol] = 0.
    
    for sub_ind,sub in iteritems(sub_dict):
        sub.update_obj(yopt)
        sub.model.optimize(precision=master.precision_sub)                
        if sub.model.Status==GRB.Status.UNBOUNDED:
            feascut = master.make_feascut(sub)
            master.model.addConstr(feascut)
        else:
            sub_obj = sub._weight*sub.model.ObjVal
            sub_objs.append(sub_obj)
            opt_sub_inds.append(sub_ind)
            ### Get non-dominated cut   
            if NONDOM:
                opt_nondom(sub, y0)
            #*** DEBUG
            if np.mod(_iter,master.print_iter)==0:
                d = sub._d
                wa = np.array([sub.model.x_dict[w.VarName] for w in sub._wa])
                wl = np.array([sub.model.x_dict[w.VarName] for w in sub._wl])
                wu = np.array([sub.model.x_dict[w.VarName] for w in sub._wu])
                waB = wa*sub._Bcsc
                cinds = waB.nonzero()[0]                                
                tk = master.model.getVarByName('tk_%s'%sub_ind)
                xl = sub._xl
                xu = sub._xu                
                waBy = LinExpr([waB[j] for j in cinds], [ys[j] for j in cinds])
                rhs2 = sum(d*wa) - waBy + sum(xl*wl) - sum(xu*wu)
                cut2 = tk >= rhs2
                cut1 = master.make_optcut(sub)
                rhs1 = cut1._rhs
                zval = z.X
                ychanged=sum(yopt-yprev)
                print("sub=%s.\tSub obj=%g\ttk=%s (value=%g)\tz=%g\tychanged=%g" % (
                    sub_ind, sub_obj, tk.VarName, tk.X, zval, ychanged))
            #**********************

    LB = z.X
    UB = sum(fy*yopt) + sum(sub_objs)
    LB = LB if abs(LB)>1e-10 else 0.
    UB = UB if abs(UB)>1e-10 else 0.
    
    if UB<bestUB:
        master.yopt = yopt
        bestUB = UB
        
    bestLB = max(bestLB,LB)
    
    gap= UB-LB
    relgap = gap / (1e-10+abs(UB))    
    
    if relgap < master.gaptol:        
        toc = time.time()-tic
        #print("iter=%d\tUB=%g\tLB=%g\tgap=%g\trelgap=%.4g%%\ttime=%g (s)" % (_iter,UB,LB,gap,relgap*100,toc))
        print("%-12.10s%-12.4g%-12.4g%-12.4g%-12.4g%-12.4g%-12.4g%-12.10g" % (
            _iter,UB,LB,bestUB,bestLB,gap,relgap*100,toc))
        print("gap<gaptol. Done.")
        break
    else:
        if not MULTICUT:
            onecut = master._z >= quicksum([master.make_optcut(sub_dict[ind])._rhs for ind in opt_sub_inds])
            master.model.addConstr(onecut)
        else:
            for k,sub_ind in enumerate(opt_sub_inds):
                tk = master.model.getVarByName('tk_%s'%sub_ind)                
                sub = sub_dict[sub_ind]
                if tk.X < sub_objs[k]:
                    cut = master.make_optcut(sub)
                    master.model.addConstr(cut)
#             onecut = master._z >= quicksum([master.make_optcut(sub_dict[ind])._rhs for ind in opt_sub_inds])
#             master.model.addConstr(onecut)

Iter        UB          LB          Best UB     Best LB     gap         relgap(%)   time(s)     
0           1e+15       -1e+15      1e+15       -1e+15      2e+15       200         0.004240036011
sub=below.	Sub obj=32666.7	tk=tk_below (value=1e+06)	z=-1e+06	ychanged=-3
sub=average.	Sub obj=32666.7	tk=tk_average (value=-1e+06)	z=-1e+06	ychanged=-3
sub=above.	Sub obj=32666.7	tk=tk_above (value=-1e+06)	z=-1e+06	ychanged=-3
1           9.8e+04     -1e+06      9.8e+04     -1e+06      1.098e+06   1120        0.01814317703
sub=below.	Sub obj=-28533.3	tk=tk_below (value=-1e+06)	z=-1e+06	ychanged=500
sub=average.	Sub obj=-42700	tk=tk_average (value=-66500)	z=-1e+06	ychanged=500
sub=above.	Sub obj=-56866.7	tk=tk_above (value=-8500)	z=-1e+06	ychanged=500
2           -5.31e+04   -1e+06      -5.31e+04   -1e+06      9.469e+05   1783        0.02980613708
sub=below.	Sub obj=-46000	tk=tk_below (value=-67866.7)	z=-136533	ychanged=0
sub=average.	Sub obj=-52666.7	tk=tk_average (value=-87333.3)	z=-136533	y

In [15]:
from cobra.solvers.gurobi_solver import status_dict

print('Status=%s'%status_dict[master.model.Status])

Status=optimal


In [16]:
for c in master.model.getConstrs():
    print(master.model.getRow(c), c.Sense, c.RHS)

(<gurobi.LinExpr: area_wheat + area_corn + area_beet>, '<', 500.0)
(<gurobi.LinExpr: z + -150.0 area_wheat + -230.0 area_corn + -260.0 area_beet + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '=', 0.0)
(<gurobi.LinExpr: 198.333333333 area_wheat + 210.0 area_corn + 240.0 area_beet + tk_average>, '>', 32666.666666666664)
(<gurobi.LinExpr: 238.0 area_wheat + 252.0 area_corn + 288.0 area_beet + tk_above>, '>', 32666.666666666664)
(<gurobi.LinExpr: 113.333333333 area_wheat + 168.0 area_corn + 192.0 area_beet + tk_below>, '>', 28133.333333333332)
(<gurobi.LinExpr: 141.666666667 area_wheat + 210.0 area_corn + 240.0 area_beet + tk_average>, '>', 28133.333333333332)
(<gurobi.LinExpr: 113.333333333 area_wheat + 168.0 area_corn + 192.0 area_beet + tk_below>, '>', 28133.333333333332)
(<gurobi.LinExpr: 141.666666667 area_wheat + 210.0 area_corn + 240.0 area_beet + tk_average>, '>', 28133.333333333332)
(<gurobi.LinExpr: 170.0 area_wheat + 252.0 area_corn + 80.0 area_beet + tk_above>, '>', -2386

## Check answer: should be 
- {'wheat':170, 'corn':80, 'beet':250}
- objval = (-)108390

In [17]:
master._z

<gurobi.Var z (value -108390.0)>

In [18]:
master._ys

[<gurobi.Var area_wheat (value 170.0)>,
 <gurobi.Var area_corn (value 80.0)>,
 <gurobi.Var area_beet (value 250.0)>]

In [19]:
#yopt = master.yopt
yopt = [y.X for y in master._ys]
x_dict = {}
for scen,sub in iteritems(master.sub_dict):
    sub.update_obj(yopt)
    sub.model.optimize()
    x_dict[scen] = {r.ConstrName:r.Pi for r in sub.model.model.getConstrs()}

In [20]:
x_dict

{'above': {'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'fixobjval': 0.0,
  'sell_beet': 6000.0,
  'sell_corn': 48.00000000000023,
  'sell_excess_beet': 9.094947017729282e-13,
  'sell_wheat': 309.99999999999966},
 'average': {'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'fixobjval': 0.0,
  'sell_beet': 5000.000000000001,
  'sell_corn': 1.7053025658242404e-13,
  'sell_excess_beet': 0.0,
  'sell_wheat': 224.99999999999972},
 'below': {'buy_beet': 0.0,
  'buy_corn': 47.99999999999986,
  'buy_wheat': 0.0,
  'fixobjval': 0.0,
  'sell_beet': 4000.0000000000005,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 139.99999999999977}}

### Check objval

In [21]:
objval = 0

for c in crops:
    a = master.model.getVarByName(area_dict[c].id)
    objval -= plantcost_dict[c]*a.X
    for scen,pk in iteritems(prob_dict):
        xsell = x_dict[scen]['sell_%s'%(c)]
        xbuy  = x_dict[scen]['buy_%s'%(c)]
        objval += pk*sellprice_dict[c]*xsell
        objval -= pk*buyprice_dict[c]*xbuy
        if c=='beet':
            xexcess = x_dict[scen]['sell_excess_%s'%(c)]
            objval += pk*excess_sell_price*xexcess

In [22]:
print('Master objval: %g, Calculated objval: %g' % (master.model.ObjVal, objval))

Master objval: -108390, Calculated objval: 108390


## Check that all constraints satisfied

In [23]:
# Total area
print('Total area constraint:', sum(yopt) <= total_area)

('Total area constraint:', True)


In [24]:
# Cattle feed crop requirement
y_dict = {y.VarName:y.X for y in master._ys}
for scen in prob_dict.keys():
    for c,req in iteritems(req_dict):
        area = y_dict['area_%s'%c]
        xsell = x_dict[scen]['sell_%s'%(c)]
        xbuy = x_dict[scen]['buy_%s'%(c)]
        lhs = yield_dict[scen][c]*area - xsell + xbuy
        rhs = req
        print('%s, %s: %g >= %g, %s'%(scen,c,lhs,rhs,lhs>=rhs))

below, beet: 0 >= 0, True
below, corn: 240 >= 240, True
below, wheat: 200 >= 200, True
average, beet: 0 >= 0, True
average, corn: 240 >= 240, True
average, wheat: 200 >= 200, True
above, beet: 9.09495e-13 >= 0, True
above, corn: 240 >= 240, True
above, wheat: 200 >= 200, True


master.model.optimize(cb_benders_multi)

yopt = np.array([y.X for y in master._ys])
xopt = []
for k in range(len(c)):
    sub = sub_dict[k]
    sub.update_obj(yopt)
    sub.model.optimize()
    xc = sub.model.model.getConstrByName('x%d'%k)
    xopt.append(xc.Pi)